# 2. Here’s the list of reviews of Chrome apps - scraped from Playstore


## Problem statement - There are times when a user writes Good, Nice App or any other positive text, in the review and gives 1-star rating. Your goal is to identify the reviews where the semantics of review text does not match rating.


Your goal is to identify such ratings where review text is good, but rating is negativeso that the support team can point this to users.



In [5]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
from nltk.stem import WordNetLemmatizer 
from textblob import TextBlob
lemmatizer = WordNetLemmatizer()

import warnings
warnings.filterwarnings('ignore')

import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [6]:
df = pd.read_csv("chrome_reviews.csv")

In [7]:
df.head()

,ID,Review URL,Text,Star,Thumbs Up,User Name,Developer Reply,Version,Review Date,App ID
0,3886,https://play.google.com/store/apps/details?id=...,This is very helpfull aap.,5,0,INDIAN Knowledge,NaN,83.0.4103.106,2020-12-19,com.android.chrome
1,3887,https://play.google.com/store/apps/details?id=...,Good,3,2,Ijeoma Happiness,NaN,85.0.4183.127,2020-12-19,com.android.chrome
2,3888,https://play.google.com/store/apps/details?id=...,Not able to update. Neither able to uninstall.,1,0,Priti D BtCFs-29,NaN,85.0.4183.127,2020-12-19,com.android.chrome
3,3889,https://play.google.com/store/apps/details?id=...,Nice app,4,0,Ajeet Raja,NaN,77.0.3865.116,2020-12-19,com.android.chrome
4,3890,https://play.google.com/store/apps/details?id=...,Many unwanted ads,1,0,Rams Mp,NaN,87.0.4280.66,2020-12-19,com.android.chrome


In [8]:
stop_words = set(stopwords.words('english'))
stop_words.remove('not')
stop_words.remove('no')

In [9]:
cleaning_text =[]
for text_review in df['Text']:
    text_review= re.sub(r'[^\w\s]','',str(text_review))
    text_review = re.sub(r'\d','',text_review)
    review_token = word_tokenize(text_review.lower().strip()) #convert reviews into lower case and strip leading and tailing spaces followed by spliting sentnece into words
    review_without_stopwords=[]
    for token in review_token:
        if token not in stop_words:
            token= lemmatizer.lemmatize(token)
            review_without_stopwords.append(token)
    cleaned_review = " ".join(review_without_stopwords)
    cleaning_text.append(cleaned_review)

In [10]:
df["cleaned_review"] = cleaning_text
Single_star_reviews = df[df.Star ==1]

In [11]:
sia = SentimentIntensityAnalyzer()
senti_list = []

for i in Single_star_reviews["cleaned_review"]:
    score = sia.polarity_scores(i)
    blob_score = TextBlob(i).sentiment.polarity
    if (score['pos'] >= 0.7):
        senti_list.append('Positive')
    else:
        senti_list.append('Negative/Neutral')
        
Single_star_reviews["sentiment"]= senti_list

In [12]:
Single_star_reviews.head()

,ID,Review URL,Text,Star,Thumbs Up,User Name,Developer Reply,Version,Review Date,App ID,cleaned_review,sentiment
2,3888,https://play.google.com/store/apps/details?id=...,Not able to update. Neither able to uninstall.,1,0,Priti D BtCFs-29,NaN,85.0.4183.127,2020-12-19,com.android.chrome,not able update neither able uninstall,Negative/Neutral
4,3890,https://play.google.com/store/apps/details?id=...,Many unwanted ads,1,0,Rams Mp,NaN,87.0.4280.66,2020-12-19,com.android.chrome,many unwanted ad,Negative/Neutral
8,3894,https://play.google.com/store/apps/details?id=...,Very bad app 😞,1,0,Akshat Bhardwaj,NaN,78.0.3904.96,2020-12-19,com.android.chrome,bad app,Negative/Neutral
9,3895,https://play.google.com/store/apps/details?id=...,Many times I tried to update its not updating....,1,0,Aditi Rathor,NaN,86.0.4240.198,2020-12-19,com.android.chrome,many time tried update not updating whenever t...,Negative/Neutral
12,3898,https://play.google.com/store/apps/details?id=...,App is not getting update and it is not gettin...,1,0,Daksh Gulati,NaN,83.0.4103.106,2020-12-19,com.android.chrome,app not getting update not getting open saying...,Negative/Neutral


In [13]:
positive_review_with_1_star = Single_star_reviews[Single_star_reviews.sentiment == 'Positive']
positive_review_with_1_star.drop("cleaned_review",axis = 1,inplace=True)

In [14]:
positive_review_with_1_star.head()

,ID,Review URL,Text,Star,Thumbs Up,User Name,Developer Reply,Version,Review Date,App ID,sentiment
101,4113,https://play.google.com/store/apps/details?id=...,Good,1,0,Sohail Soomro,NaN,74.0.3729.136,2020-12-21,com.android.chrome,Positive
158,4143,https://play.google.com/store/apps/details?id=...,Good,1,0,Md Rubel khan,NaN,NaN,2020-12-21,com.android.chrome,Positive
291,5251,https://play.google.com/store/apps/details?id=...,Ok,1,0,Rajesh Prabhu,NaN,76.0.3809.111,2020-12-21,com.android.chrome,Positive
310,5842,https://play.google.com/store/apps/details?id=...,Best,1,0,Vasava Govindbhai,NaN,67.0.3396.87,2020-12-22,com.android.chrome,Positive
312,5844,https://play.google.com/store/apps/details?id=...,Good,1,0,Prem Kumar,NaN,80.0.3987.99,2020-12-22,com.android.chrome,Positive


In [15]:
positive_review_with_1_star.to_csv('output.csv')